In [1]:
%load_ext autoreload
%autoreload 2
import adanet

In [2]:
OUTPUT_DIR = './tmp/adanet'
datadir = './data'
is_tensorboard = True

In [3]:
# this will kill the processes for Tensorboard
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill

# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill

kill: (25461)kill: (25467)

In [4]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset)
print(x_train.shape, x_test.shape)

Using TensorFlow backend.


p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 224, 224, 3) (47, 224, 224, 3)


In [5]:
import p8_util_config
p8_util_config.ADANET_MAX_ITERATION_STEPS

3

In [6]:
import p8_util_config
import NNGenerator

nn_type = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type']
my_config = p8_util.make_config(nn_type,output_dir=OUTPUT_DIR, is_restored=False)

nClasses = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type']

estimator = adanet.Estimator(
    head=p8_util_config.dict_adanet_config['adanet_tf_head'],
    
    subnetwork_generator=NNGenerator.NNGenerator(p8_util_config.dict_adanet_config),
            
    adanet_lambda=p8_util_config.dict_adanet_config['adanet_lambda'],#0.0,#.015,
    
    max_iteration_steps=p8_util_config.ADANET_MAX_ITERATION_STEPS,
    
    evaluator=adanet.Evaluator(
        input_fn=p8_util.input_fn("train", x_train, y_train\
                          , p8_util_config.NUM_EPOCHS
                          , tuple_dimension=tuple_dimension\
                          , batch_size=p8_util_config.BATCH_SIZE),
        steps=None),
    config=  my_config)

*** INFO : MyGenerator : instantiation DONE!
INFO:tensorflow:Using config: {'_model_dir': './tmp/adanet/CNN', '_tf_random_seed': 42, '_save_summary_steps': 5, '_save_checkpoints_steps': 5, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f04d26f13c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
import p8_util_config
print("Global steps= {}".format(p8_util_config.TRAIN_STEPS))
print("MAX Adanet steps= {}".format(p8_util_config.ADANET_MAX_ITERATION_STEPS))

Global steps= 9
MAX Adanet steps= 3


In [8]:
import time
import tensorflow as tf

start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(
        input_fn= p8_util.input_fn("train"
                           , x_train
                           , y_train, p8_util_config.NUM_EPOCHS \
                           , tuple_dimension=tuple_dimension\
                           , batch_size=p8_util_config.BATCH_SIZE),
        max_steps=p8_util_config.TRAIN_STEPS),
    eval_spec=tf.estimator.EvalSpec(
        input_fn= p8_util.input_fn("test"
                                   , x_test
                                   , y_test
                                   , p8_util_config.NUM_EPOCHS\
                                   , tuple_dimension=tuple_dimension\
                                   , batch_size=p8_util_config.BATCH_SIZE),
        steps=None,
        start_delay_secs=1,
        throttle_secs=1,  
    )
)
end_time = time.time()

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 5 or save_checkpoints_secs None.
INFO:tensorflow:Beginning training AdaNet iteration 0
INFO:tensorflow:Calling model_fn.

*** NNGenerator : layers= (0,1)

*** NNAdaNetBuilder : NN Type=CNN

*** NNAdaNetBuilder : NN Type=CNN
INFO:tensorflow:Building iteration 0
INFO:tensorflow:Building subnetwork 'CNN_linear'


*** build_subnetwork() : NN type= CNN
INFO:tensorflow:Building subnetwork 'CNN_layer_1'


*** build_subnetwork() : NN type= CNN
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./

INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_CNN_layer_1/weighted_subnetwork_0/subnetwork/batch_normalization_2/moving_mean; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_CNN_layer_1/weighted_subnetwork_0/subnetwork/batch_normalization_2/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_CNN_layer_1/weighted_subnetwork_0/subnetwork/conv2d_2/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_CNN_layer_1/weighted_subnetwork_0/subnetwork/conv2d_2/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_CNN_layer_1/weighted_subnetwork_0/subnetwork/batch_normalization_3/gamma; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_CNN_layer_1/weighted_subnetwork_0/subnetwork/batch_normalization_3/beta; prev_var_name: U

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6: ./tmp/adanet/CNN/model.ckpt-6
INFO:tensorflow:Loss for final step: 1.1019964.
INFO:tensorflow:Finished training Adanet iteration 1
INFO:tensorflow:Beginning bookkeeping phase for iteration 1
INFO:tensorflow:Importing architecture from ./tmp/adanet/CNN/architecture-0.txt: ['0:CNN_layer_1'].

*** NNGenerator : layers= (0,1)

*** NNAdaNetBuilder : NN Type=CNN

*** NNAdaNetBuilder : NN Type=CNN
INFO:tensorflow:Rebuilding iteration 0
INFO:tensorflow:Building subnetwork 'CNN_layer_1'


*** build_subnetwork() : NN type= CNN

*** NNGenerator : layers= (1,2)

*** NNAdaNetBuilder : NN Type=CNN

*** NNAdaNetBuilder : NN Type=CNN
INFO:tensorflow:Building iteration 1
INFO:tensorflow:Building subnetwork 'CNN_layer_1'


*** build_subnetwork() : NN type= CNN
INFO:tensorflow:Building subnetwork 'CNN_layer_2'


*** build_subnetwork() : NN type= CNN
INFO:tensorflow:Starting ensemble evaluation for iteration 1
INFO:tensorflow:Restoring pa

INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_CNN_layer_2/weighted_subnetwork_1/subnetwork/batch_normalization/gamma; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_CNN_layer_2/weighted_subnetwork_1/subnetwork/batch_normalization/beta; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_CNN_layer_2/weighted_subnetwork_1/subnetwork/batch_normalization/moving_mean; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_CNN_layer_2/weighted_subnetwork_1/subnetwork/batch_normalization/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_CNN_layer_2/weighted_subnetwork_1/subnetwork/conv2d/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_CNN_layer_2/weighted_subnetwork_1/subnetwork/conv2d/bias; prev_var_name: Unchanged
INF

INFO:tensorflow:Saving checkpoints for 9 into ./tmp/adanet/CNN/model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Importing architecture from ./tmp/adanet/CNN/architecture-1.txt: ['0:CNN_layer_1', '1:CNN_layer_2'].

*** NNGenerator : layers= (0,1)

*** NNAdaNetBuilder : NN Type=CNN

*** NNAdaNetBuilder : NN Type=CNN
INFO:tensorflow:Rebuilding iteration 0
INFO:tensorflow:Building subnetwork 'CNN_layer_1'


*** build_subnetwork() : NN type= CNN

*** NNGenerator : layers= (1,2)

*** NNAdaNetBuilder : NN Type=CNN

*** NNAdaNetBuilder : NN Type=CNN
INFO:tensorflow:Rebuilding iteration 1
INFO:tensorflow:Building subnetwork 'CNN_layer_2'


*** build_subnetwork() : NN type= CNN

*** NNGenerator : layers= (2,3)

*** NNAdaNetBuilder : NN Type=CNN

*** NNAdaNetBuilder : NN Type=CNN
INFO:tensorflow:Building iteration 2
INFO:tensorflow:Building subnetwork 'CNN_layer_2'


*** build_subnetwork() : NN type= CNN
INFO:tensorflow:Building subnetwork 'CNN_layer_3'


*** build_subnetwork() : NN 

In [9]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("\n{}: {}".format(title, results[key]))



------------------------------------------------
Time (sec) 706.9577541351318

ACCURACY: 0.2978723347187042

ACCURACY/ADANET/ADANET_WEIGHTED_ENSEMBLE: 0.2978723347187042

ACCURACY/ADANET/SUBNETWORK: 0.3404255211353302

ACCURACY/ADANET/UNIFORM_AVERAGE_ENSEMBLE: 0.2978723347187042

ARCHITECTURE/ADANET/ENSEMBLES: b'\n}\n>adanet/iteration_2/ensemble_t2_CNN_layer_2/architecture/adanetB1\x08\x07\x12\x00B+| CNN_layer_1 | CNN_layer_2 | CNN_layer_2 |J\x08\n\x06\n\x04text'

AVERAGE_LOSS: 1.0973129272460938

AVERAGE_LOSS/ADANET/ADANET_WEIGHTED_ENSEMBLE: 1.0973129272460938

AVERAGE_LOSS/ADANET/SUBNETWORK: 1.0983067750930786

AVERAGE_LOSS/ADANET/UNIFORM_AVERAGE_ENSEMBLE: 1.0973129272460938

LOSS: 1.1052461862564087

LOSS/ADANET/ADANET_WEIGHTED_ENSEMBLE: 1.1052461862564087

LOSS/ADANET/SUBNETWORK: 1.0991201400756836

LOSS/ADANET/UNIFORM_AVERAGE_ENSEMBLE: 1.1052465438842773

GLOBAL_STEP: 9


In [10]:
#import tensorflow as tf
#help(tf.estimator.Estimator)

In [11]:
def ensemble_architecture(result):
  """Extracts the ensemble architecture from evaluation results."""

  architecture = result["architecture/adanet/ensembles"]
  # The architecture is a serialized Summary proto for TensorBoard.
  summary_proto = tf.summary.Summary.FromString(architecture)
  return summary_proto.value[0].tensor.string_val[0]

In [12]:
print("Architecture: {} / Accuracy= {} /Loss= {}".format(ensemble_architecture(results)\
                                                  , results['accuracy'], results['loss']))

Architecture: b'| CNN_layer_1 | CNN_layer_2 | CNN_layer_2 |' / Accuracy= 0.2978723347187042 /Loss= 1.1052461862564087


In [13]:
# Architecture: b'| DNN_layer_1 | DNN_layer_2 | DNN_layer_3 |' : accuracy = 40% / Loss = 1.0883579
# Architecture: b'| CNNBase_layer_1 |' / Accuracy= 0.40425533056259155 /Loss= 1.081491470336914
# Architecture: b'| CNNBase_linear | CNNBase_layer_1 |' / Accuracy= 0.42553192377090454 /Loss= 1.1250361204147339

In [14]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')

In [15]:
[(i,i%2) for i in range(0,10)]

[(0, 0),
 (1, 1),
 (2, 0),
 (3, 1),
 (4, 0),
 (5, 1),
 (6, 0),
 (7, 1),
 (8, 0),
 (9, 1)]

In [16]:
1%2

1

In [17]:
2%2

0

In [18]:
3%2

1

In [19]:
4%2

0